In [1]:
%run -i rename.py

10560667648 bytes of graphical memory available.
11551571968 bytes of graphical memory available.


In [ ]:
from torchvision.io import read_image, write_jpeg
filename = '../upload/v/{}.png'
video = torch.stack([read_image(filename.format(str(i).zfill(8))) for i in range(62)])
# video, audio, metadata = read_video('realshort.mp4')
video.shape

In [ ]:
def normalize(x, centralize=False, normalize=False, val_range=255.0):
    if centralize:
        x = x - val_range / 2
    if normalize:
        x = x / val_range

    return x


def normalize_reverse(x, centralize=False, normalize=False, val_range=255.0):
    if normalize:
        x = x * val_range
    if centralize:
        x = x + val_range / 2

    return x.clamp(0, val_range - 1)
val_range = 2.0 ** 8
inp = video.to(dtype=config.dtype(), device=config.device())
inp = normalize(inp, centralize=para.centralize, normalize=para.normalize, val_range=val_range).unsqueeze(0)
output = normalize_reverse(m(inp).squeeze(0), centralize=para.centralize, normalize=para.normalize, val_range=val_range)
video = output.to(dtype=torch.uint8, device=torch.device('cpu'))
video.shape

In [ ]:
filename = '../download/target/{}.png'
for i, img in enumerate(video):
  write_jpeg(img.to(dtype=torch.uint8), filename.format(str(i + 2).zfill(2)), quality=98)

In [ ]:
write_video('../download/target.mp4', video_array=video, fps=24)